# Popular Data Science Questions On Stack Exchange

In this project we will be working with data from Stack Exchange, a question-answer forum focused on programming related topics. Our task is to determine the best content to write posts about within the realm of Data Science. Stack Exchange has a Data Science focused microsite that we will help us focus our analysis.

Stack Exchange makes its data publicly available via the Stack Exchange Data Explorer (SEDE), which allows SQL queries of its database and exportation to .csv files. We will be analyzing a .csv file containing data from Posts database over the year 2019.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read in the dataframe
data = pd.read_csv('2019_questions.csv')

In [3]:
# show a sample of the data available
data.head()

,Id,CreationDate,Score,ViewCount,Tags,AnswerCount,FavoriteCount
0,44419,2019-01-23 09:21:13,1,21,<machine-learning><data-mining>,0,NaN
1,44420,2019-01-23 09:34:01,0,25,<machine-learning><regression><linear-regressi...,0,NaN
2,44423,2019-01-23 09:58:41,2,1651,<python><time-series><forecast><forecasting>,0,NaN
3,44427,2019-01-23 10:57:09,0,55,<machine-learning><scikit-learn><pca>,1,NaN
4,44428,2019-01-23 11:02:15,0,19,<dataset><bigdata><data><speech-to-text>,0,NaN


In [4]:
# total rows and columns in the dataframe
data.shape

(8839, 7)

In [8]:
# more info
data.describe()

,Id,Score,ViewCount,AnswerCount,FavoriteCount
count,8839.000000,8839.000000,8839.000000,8839.000000,1407.000000
mean,54724.172870,0.870687,171.548026,0.787985,1.184790
std,6507.618509,1.410255,772.813626,0.851146,0.982766
min,43363.000000,-2.000000,2.000000,0.000000,0.000000
25%,48917.500000,0.000000,22.000000,0.000000,1.000000
50%,54833.000000,1.000000,40.000000,1.000000,1.000000
75%,60674.500000,1.000000,98.000000,1.000000,1.000000
max,65675.000000,45.000000,33203.000000,9.000000,16.000000


In [7]:
# data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8839 entries, 0 to 8838
Data columns (total 7 columns):
Id               8839 non-null int64
CreationDate     8839 non-null object
Score            8839 non-null int64
ViewCount        8839 non-null int64
Tags             8839 non-null object
AnswerCount      8839 non-null int64
FavoriteCount    1407 non-null float64
dtypes: float64(1), int64(4), object(2)
memory usage: 483.5+ KB


We have 7 columns of datapoints and 8839 rows of data entries. The data is comprised of integer, string object, and float types. There is a high degree of completeness to the data - the exception being the Favorite count column, which only has 1407 non-null values.

From their website we know that each column roughly means the following:

- Id: An identification number for the post.
- CreationDate: The date and time of creation of the post.
- Score: The post's score.
- ViewCount: How many times the post was viewed.
- Tags: What tags were used.
- AnswerCount: How many answers the question got (only applicable to question posts).
- FavoriteCount: How many times the question was favored (only applicable to question posts).

The Tags column will be critical for our analysis, as it tells us subjectively what each post is about. We will have to find a way to parse out the tags for each post, of which there can be none or several.

The Score, ViewCount, AnswerCount, and FavoriteCount columns will give us engagement metrics.

The CreationDate column will give us a sense of any seasonality.

# Data Cleaning

Let's start with an evaluation of our FavoriteCount column.

In [10]:
# create a frequency table of values
data['FavoriteCount'].value_counts(dropna=False).sort_index(ascending=False)

 16.0       1
 11.0       1
 8.0        1
 7.0        4
 6.0        4
 5.0        8
 4.0       12
 3.0       43
 2.0      205
 1.0      953
 0.0      175
NaN      7432
Name: FavoriteCount, dtype: int64

In [11]:
# replace NaN with zeros
data['FavoriteCount'].fillna(0, inplace = True)

In [12]:
# convert series to integer
data['FavoriteCount'] = data['FavoriteCount'].astype(int)

Our CreationDate column needs to be converted to DateTime format.

In [13]:
# convert to datetime
data['CreationDate'] = pd.to_datetime(data['CreationDate'])

### Tags

The Tags data should be converted to a string format that is conducive to analysis techniques. We will change from \<tag> to comma separated values so we can use list manipulation techniques.

In [14]:
data['Tags'].head()

0                      <machine-learning><data-mining>
1    <machine-learning><regression><linear-regressi...
2         <python><time-series><forecast><forecasting>
3                <machine-learning><scikit-learn><pca>
4             <dataset><bigdata><data><speech-to-text>
Name: Tags, dtype: object

In [15]:
# use pandas string methods to turn these into lists
data['Tags'] = data['Tags'].str.replace('<','').str.replace('>',',')

In [17]:
# confirming the change
data['Tags'].head()

0                        machine-learning,data-mining,
1    machine-learning,regression,linear-regression,...
2             python,time-series,forecast,forecasting,
3                   machine-learning,scikit-learn,pca,
4                 dataset,bigdata,data,speech-to-text,
Name: Tags, dtype: object

### Most Popular Tags

In [20]:
# create empty dictionary
tag_count = {}
# loop through the rows in each series then loop through the list in tags and create a frequency table dictionary
for tag_list in data['Tags']:
    for tag in tag_list:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1

In [21]:
# turn the dictionary into a dataframe
tags = pd.DataFrame(list(tag_count.items()), columns = ['tag','count'])

In [22]:
# show the top values 
tags.sort_values('count',ascending=False).head()

,tag,count
0,,8839
232,machine-learning,2693
105,python,1814
306,deep-learning,1220
182,neural-network,1055


In [23]:
# turn no posts with no tags to np.NaN
tags.iloc[2:3,0:1] = np.NaN

In [28]:
# show the top 20 values
top20_tags_frequency = tags.sort_values('count',ascending=False).head(20)
top20_tags_frequency

,tag,count
0,,8839
232,machine-learning,2693
105,python,1814
306,deep-learning,1220
182,neural-network,1055
36,keras,935
305,classification,685
77,tensorflow,584
465,scikit-learn,540
524,nlp,493


In [25]:
# create empty dictionary
tag_views = {}

# iterrows allows us to loop through each row in a dataframe
for index, row in data.iterrows():
    # loop through the list of each row
    for tag in row['Tags']:
        # check if the tag is in the dictionary and add the values
        if tag in tag_views:
            tag_views[tag] += row['ViewCount']
        else:
            tag_views[tag] = row['ViewCount']

In [29]:
# turn the dictionary into a dataframe
tag_tot_views = pd.DataFrame(list(tag_views.items()), columns = ['tag','views'])

In [30]:
# turn the empty tag to np.nan
tag_tot_views['tag'][tag_tot_views['tag'] == ''] = np.nan

/dataquest/system/env/python3/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [31]:
# show the top 20 tags based on views
top20_tags_views = tag_tot_views.sort_values(by = 'views',ascending = False).head(20)
top20_tags_views

,tag,views
0,NaN,1516313
105,python,537585
232,machine-learning,388499
36,keras,268608
306,deep-learning,233628
91,pandas,201787
182,neural-network,185367
465,scikit-learn,128110
77,tensorflow,121369
305,classification,104457


In [36]:
# define the format we want the subplots to follow
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (20,7))
# plot the two dataframes
top20_tags_frequency.plot(kind = 'barh', x = 'tag', y = 'count', ax = axes[0], subplots = True, legend = False, title = 'Top 20 Tags by Usage & Views')
top20_tags_views.plot(kind = 'barh', x = 'tag', y = 'views', ax = axes[1], legend = False, subplots = True)
plt.show()